In [1]:
import random 
from IPython.display import clear_output

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

In [2]:
class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank + ' of ' + self.suit

In [3]:
class Deck:
    
    def __init__(self):
        # Note this only happens once upon creation of a new Deck
        self.all_cards = [] 
        for suit in suits:
            for rank in ranks:
                # This assumes the Card class has already been defined!
                self.all_cards.append(Card(suit,rank))
                
    def shuffle(self):
        # Note this doesn't return anything
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        # Note we remove one card from the list of all_cards
        return self.all_cards.pop()   

In [4]:
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card):
        self.cards.append(card)
        self.value += (values[card.rank])
        if card.rank == "Ace":
            self.aces += 1
    
    def adjust_for_ace(self):
        while self.aces >0:
            v = int(input("Select if you want the ace to be considered as 1 or 11"))
            if v == 1:
                self.value -=10
            self.aces -= 1
    def dealer_adjust_for_ace(self):
        while self.aces >0:
            if self.value > 21:
                self.value -=10
            self.aces -= 1
    def get_value(self):
        return self.value


In [5]:
class Chips:
    
    def __init__(self, total=100, bet=1):
        self.total = total  
        self.bet = bet
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
    def blackjack(self):
        print("This is a BlackJack!!")
        self.total += 2*self.bet
    

In [6]:
class Player(Hand):
    def hit(self, deck):
        card = deck.deal_one()
        print("The given card is", card)
        self.add_card(card)
        self.adjust_for_ace()
        bust =self.busts()
        return bust
    
    def hit_or_stand(self, deck):
        play = True
        no_bust = True
        while play:
            if no_bust:
                print("Your current value is ", self.get_value())
                choice = input("Do you want to Hit(H) or Stay(S)?")
                if choice not in ["H","S"]:
                    clear_output()
                    print("Choose a valid option")
                elif choice == "S":
                    print("The player stopped with a value of ", self.get_value())
                    break
                else:
                    no_bust = self.hit(deck)
            else:
                play = False
        return no_bust
    
    def busts(self):
        if self.get_value() > 21:
            print("Player got over 21. The player has lost the game")
            return False
        else:
            return True
        
    def show_card(self):
        for i in self.cards:
            print(i, end = " ")

In [7]:
class Dealer(Hand):
    def hit_dealer(self, deck):
        card = deck.deal_one()
        print("The given card is", card)
        self.add_card(card)
        self.dealer_adjust_for_ace()
        bust = self.dealer_busts()
        return bust 
    
    def dealer_hit_or_stand(self, deck, player):
        play = True
        no_bust = True
        while play:
            if no_bust:
                print("Dealer current value is ", self.get_value())
                if dealer.get_value() < 17:
                     no_bust = self.hit_dealer(deck)
                elif dealer.get_value() < player.get_value():
                    no_bust = self.hit_dealer(deck)
                else:
                    print("The Dealer stopped with a value of ", self.get_value())
                    break
            else:
                play = False
        return no_bust
    
    def show_card(self):
        for i in self.cards:
            print(i, end = " ")
    def show_dealer(self):
        print("hidden", end = " ")
        print(self.cards[-1])
        
    def dealer_busts(self):
        if self.get_value() > 21:
            print("The dealer has got over 21. The player has won the game")
            return False
        else:
            return True
    

In [8]:
def take_bet(chip):
    choice = "wrong"
    while choice != "valid":
        value = int(input("Select a bet value:"))
        if value not in range(chip.total+1):
            clear_output()
            print("Please choose a valid option")
        else:
            choice = "valid"
    print("Player has bet {} for this hand".format(value))
    return value

def wins(play_value, dealer_value):
    if play_value > dealer_value:
        print("Player win his bet")
        return True
    else:
        print("Player lost his bet")
        return False
    

In [9]:
game_on = True
deck = Deck()
player_chips = Chips()
while game_on:
    if len(deck.all_cards) < 10:
        print("Getting a new deck of cards")
        deck = Deck()
    deck.shuffle()
    player = Player()
    dealer = Dealer()
    print("Your current balance is ", player_chips.total)
    bet = take_bet(player_chips)
    player_chips.bet = bet
    for i in range(2):
        player.add_card(deck.deal_one())
        dealer.add_card(deck.deal_one())
    print("The Player Hand is:")
    player.show_card()
    print("\n")
    print("The Dealer Hand is:")
    dealer.show_dealer()
    print("\n")

    player_game = player.hit_or_stand(deck)
    if player_game == False:
        player_chips.lose_bet()
    else:
        print("\n")

        print("The revealed dealer hand is:")
        dealer.show_card()
        print("\n")
        dealer_game = dealer.dealer_hit_or_stand(deck, player)
        if dealer_game == False:
            if len(player.cards) == 2 and player.value ==21:
                player_chips.blackjack()
            else:
                player_chips.win_bet()
    if player_game== True and dealer_game == True:
        game = wins(player.get_value(), dealer.get_value())
        if game:
            if len(player.cards) == 2 and player.value ==21:
                player_chips.blackjack()
            else:
                player_chips.win_bet()
        else:
            player_chips.lose_bet()
    print("Your current balance is: ", player_chips.total)
    if player_chips.total >0:
        p = input("Do you wanna play another round? (Y or N)")
        if p == "N":
            game_on = False
        else:
            clear_output()
    else:
        print("You lost all your money, you schmuck")
        game_on = False

Your current balance is  100


Select a bet value: 50


Player has bet 50 for this hand
The Player Hand is:
King of Hearts Eight of Hearts 

The Dealer Hand is:
hidden Queen of Hearts


Your current value is  18


Do you want to Hit(H) or Stay(S)? S


The player stopped with a value of  18


The revealed dealer hand is:
Three of Clubs Queen of Hearts 

Dealer current value is  13
The given card is Seven of Clubs
Dealer current value is  20
The Dealer stopped with a value of  20
Player lost his bet
Your current balance is:  50


Do you wanna play another round? (Y or N) N
